In [19]:
# libraries
import re
import pandas as pd

In [20]:
# combine csv files
posts1 = pd.read_csv(
    "/Users/seshat/Documents/GitHub/labor_sentiment_analysis/data/raw/posts1.csv"
)
posts2 = pd.read_csv(
    "/Users/seshat/Documents/GitHub/labor_sentiment_analysis/data/raw/posts2.csv"
)
posts = pd.concat([posts1, posts2], ignore_index=True)

comments1 = pd.read_csv(
    "/Users/seshat/Documents/GitHub/labor_sentiment_analysis/data/raw/comments1.csv"
)
comments2 = pd.read_csv(
    "/Users/seshat/Documents/GitHub/labor_sentiment_analysis/data/raw/comments2.csv"
)
comments = pd.concat([comments1, comments2], ignore_index=True)
comments = comments.drop_duplicates()

In [21]:
# clean
posts = posts.drop_duplicates(keep="first")


In [22]:
posts["text"] = posts["title"].fillna("") + " " + posts["content"].fillna("")
posts = posts[["subreddit", "post_id", "text", "score", "created"]]

In [23]:
comments = comments.drop_duplicates(keep="first")

In [24]:
comments = comments[["subreddit", "post_id", "content", "score", "created"]]
comments = comments.rename(columns={"content": "text"})

In [25]:
# combine 
reddit = pd.concat([posts, comments], ignore_index=True)

In [26]:
def clean_text(text):
    """Makes text lowercase, removes special characters, and removes extra spaces."""
    if isinstance(text, str):  # Ensure the text is a string
        # Lowercase
        text = text.lower()
        # Remove special characters
        text = re.sub(r"[^a-z\s]", "", text)
        # Remove extra spaces
        text = re.sub(r"\s+", " ", text).strip()
    else:
        text = ""
    return text

In [27]:
# clean
reddit["text"] = reddit["text"].apply(clean_text)

In [28]:
# datetime column
# Convert the 'created' column to datetime
reddit = reddit.rename(columns={"created": "datetime"})
reddit["datetime"] = pd.to_datetime(reddit["datetime"])

In [29]:
reddit["year"] = reddit["datetime"].dt.year
reddit["month"] = reddit["datetime"].dt.month
reddit["day"] = reddit["datetime"].dt.day

reddit = reddit[["subreddit", "post_id", "text", "score", "year", "month", "day"]]

In [30]:
reddit = reddit.fillna("")

In [31]:
# filter text for keywords
keywords = ["union", "strike"]
pattern = "|".join(keywords)

reddit = reddit[reddit["text"].str.contains(pattern, case=False, na=False)]

In [32]:
# Stratified sampling by subreddit
reddit = reddit.dropna(subset=["subreddit"])
reddit_sampled = (
    reddit.groupby("subreddit", group_keys=False)
    .apply(lambda x: x.sample(n=min(len(x), 100), random_state=42))
    .reset_index(drop=True)
)

# Check if enough rows exist for final sampling
if len(reddit_sampled) >= 2000:
    # Randomly sample 2000 rows
    reddit_sampled = reddit_sampled.sample(n=2000, random_state=42)
else:
    # Sample all rows if less than 2000 are available
    print(f"Total rows available: {len(reddit_sampled)}")
    reddit_sampled = reddit_sampled.sample(n=len(reddit_sampled), random_state=42)

print(f"Final sampled_reddit size: {len(reddit_sampled)}")

Total rows available: 1322
Final sampled_reddit size: 1322


/var/folders/nx/hb3zcj293r197l8g8x2s58900000gn/T/ipykernel_19716/3004898917.py:5: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda x: x.sample(n=min(len(x), 100), random_state=42))


In [33]:
# export data
reddit_sampled.to_csv(
    "/Users/seshat/Documents/GitHub/labor_sentiment_analysis/data/train/reddit_train.csv",
    index=False,
)

reddit.to_csv(
    "/Users/seshat/Documents/GitHub/labor_sentiment_analysis/data/master/reddit.csv",
    index=False,
)

posts.to_csv(
    "/Users/seshat/Documents/GitHub/labor_sentiment_analysis/data/master/posts.csv",
    index=False,
)

comments.to_csv(
    "/Users/seshat/Documents/GitHub/labor_sentiment_analysis/data/master/comments.csv",
    index=False,
)